In [1]:
import pandas as pd

In [30]:
train = pd.read_csv('sample_data/itunes-amazon/train.csv')
validation = pd.read_csv('sample_data/itunes-amazon/validation.csv')
test = pd.read_csv('sample_data/itunes-amazon/test.csv')

In [3]:
train_positives = train[train['label']==1]
train_negatives = train[train['label']==0]

In [31]:
validation_positives = validation[validation['label']==1]
validation_negatives = validation[validation['label']==0]

In [25]:
def get_price_distribution(df):
    prices = {}
    for idx,row in df.iterrows():
        current_prices = row['left_Price']+ " "+ row['right_Price']
        if current_prices in prices:
            prices[current_prices] += 1
        else:
            prices[current_prices] = 1
    return prices

In [28]:
price_negatives = get_price_distribution(train_negatives)
price_negatives

{'$ 1.29 $ 1.29': 153,
 'Album Only $ 1.29': 20,
 '$ 1.29 $ 0.99': 18,
 '$ 1.99 $ 1.29': 25,
 '$ 1.29 $ 0.69': 3,
 '$ 0.99 $ 0.99': 11,
 '$ 1.99 $ 0.99': 3,
 '$ 0.99 $ 1.29': 11,
 '$ 0.99 $ 0.89': 2}

In [29]:
price_positives = get_price_distribution(train_positives)
price_positives

{'$ 0.99 $ 0.99': 17,
 '$ 1.29 $ 1.29': 58,
 '$ 1.99 $ 1.29': 1,
 '$ 1.29 $ 0.89': 1}

In [32]:
validation_positives = get_price_distribution(validation_positives)
validation_positives

{'$ 1.29 $ 1.29': 18,
 '$ 1.29 $ 0.89': 1,
 '$ 0.99 $ 0.99': 2,
 'Album Only $ 1.29': 1,
 '$ 1.99 $ 1.29': 1,
 'Album Only $ 0.99': 1}

In [33]:
price_neg_valid = get_price_distribution(validation_negatives)
price_neg_valid

{'$ 1.29 $ 1.29': 50,
 'Album Only $ 1.29': 8,
 '$ 1.29 $ 0.99': 10,
 '$ 1.99 $ 1.29': 8,
 '$ 0.99 $ 0.99': 5,
 '$ 0.99 $ 1.29': 1,
 '$ 0.99 $ 0.89': 1,
 '$ 1.29 $ 0.89': 1}

### Evaluation changing prices

In [34]:
import deepmatcher as dm
hybrid_model = dm.MatchingModel(attr_summarizer='hybrid')

In [35]:
hybrid_model.load_state('models/hybrid_model.pth')
hybrid_model.cuda()

MatchingModel(
  (attr_summarizers): ModuleMap(
    (Song_Name): Hybrid(
      (word_contextualizer): RNN(
        (rnn_groups): ModuleList(
          (0): GRU(300, 150, batch_first=True, bidirectional=True)
        )
        (dropouts): ModuleList(
          (0): Dropout(p=0)
        )
        (bypass_networks): ModuleList(
          (0): None
        )
        (input_dropout): NoMeta(
          (module): Dropout(p=0)
        )
      )
      (word_comparator): Attention(
        (alignment_networks): ModuleList(
          (0): AlignmentNetwork(
            (transform): Transform(
              (transforms): ModuleList(
                (0): Linear(in_features=300, out_features=300, bias=True)
                (1): Linear(in_features=300, out_features=300, bias=True)
              )
              (bypass_networks): ModuleList(
                (0): Bypass(
                  (highway_gate): Linear(in_features=300, out_features=300, bias=True)
                )
                (1): Bypass(


In [144]:
val_pos = dm.data.process_unlabeled('sample_data/itunes-amazon/validation_positives.csv',trained_model=hybrid_model,
                                     ignore_columns=['id','label'])

In [145]:
predictions = hybrid_model.run_prediction(val_pos,output_attributes=True)

===>  PREDICT Epoch 2
Finished Epoch 2 || Run Time:    0.1 | Load Time:    0.1 || F1:   0.00 | Prec:   0.00 | Rec:   0.00 || Ex/s:   0.00



In [146]:
predictions[predictions['match_score']<=0.5]

,match_score,label,left_Song_Name,left_Artist_Name,left_Album_Name,left_Genre,left_Price,left_CopyRight,left_Time,left_Released,right_Song_Name,right_Artist_Name,right_Album_Name,right_Genre,right_Price,right_CopyRight,right_Time,right_Released
id,,,,,,,,,,,,,,,,,,
276,0.299670,1,Whateva ( feat . Monty ),Fetty Wap,Fetty Wap ( Deluxe ),"Hip-Hop/Rap , Music , Rap",Album Only,2015 300 Entertainment/RGF Productions,5:11,NaN,Whateva ( feat . Monty ) [ Explicit ],Fetty Wap,Fetty Wap ( Deluxe ) [ Explicit ],Rap & Hip-Hop,$ 1.29,"( C ) 2000 Mute Records Ltd. , a BMG Company ,...",5:11,"September 25 , 2015"
353,0.130318,1,Remember You ( feat . The Weeknd ),Wiz Khalifa,O.N.I.F.C. ( Deluxe Version ),"Hip-Hop/Rap , Music , East Coast Rap , Hardcor...",$ 1.99,2012 Atlantic Recording Corporation for the Un...,5:20,4-Dec-12,Remember You ( feat . The Weeknd ) [ Explicit ],Wiz Khalifa,O.N.I.F.C. [ Explicit ],Rap & Hip-Hop,$ 1.29,2012 Atlantic Recording Corporation for the Un...,4:49,"December 4 , 2012"


In [147]:
def alter_price(df,left_pr,right_pr):
    df['left_Price'] = left_pr
    df['right_Price'] = right_pr
    return df

In [163]:
! rm sample_data/itunes-amazon/val_positives_altered_price.csv

In [164]:
! rm sample_data/itunes-amazon/test_positives_altered_price.csv

In [165]:
test_positives = pd.read_csv('sample_data/itunes-amazon/test_positives.csv')

In [166]:
val_pos = pd.read_csv('sample_data/itunes-amazon/validation_positives.csv')

In [170]:
test_pos_altered = alter_price(test_positives,'Album Only','$ 1.29')

In [171]:
val_pos_altered = alter_price(val_pos,'$ 1.99','$ 1.29')

In [172]:
val_pos_altered.to_csv('sample_data/itunes-amazon/val_positives_altered_price.csv',index=False)

In [173]:
test_pos_altered.to_csv('sample_data/itunes-amazon/test_positives_altered_price.csv',index=False)

In [174]:
val_pos_altered_price = dm.data.process_unlabeled('sample_data/itunes-amazon/val_positives_altered_price.csv',
                                                 trained_model=hybrid_model,ignore_columns=['id','label'])

In [175]:
test_pos_altered_price = dm.data.process_unlabeled('sample_data/itunes-amazon/test_positives_altered_price.csv'
                                                   ,trained_model=hybrid_model,ignore_columns=['id','label'])

In [176]:
new_preds = hybrid_model.run_prediction(test_pos_altered_price,output_attributes=True)

===>  PREDICT Epoch 2
Finished Epoch 2 || Run Time:    0.2 | Load Time:    0.1 || F1:   0.00 | Prec:   0.00 | Rec:   0.00 || Ex/s:   0.00



In [177]:
new_preds_val = hybrid_model.run_prediction(val_pos_altered_price,output_attributes = True)

===>  PREDICT Epoch 2
Finished Epoch 2 || Run Time:    0.1 | Load Time:    0.1 || F1:   0.00 | Prec:   0.00 | Rec:   0.00 || Ex/s:   0.00



In [178]:
new_preds[new_preds['match_score']<=0.5]

,match_score,label,left_Song_Name,left_Artist_Name,left_Album_Name,left_Genre,left_Price,left_CopyRight,left_Time,left_Released,right_Song_Name,right_Artist_Name,right_Album_Name,right_Genre,right_Price,right_CopyRight,right_Time,right_Released
id,,,,,,,,,,,,,,,,,,
411,0.491081,1,Better Than Words,One Direction,Midnight Memories,"Pop , Music , Rock , Pop/Rock , Teen Pop , Dance",Album Only,2013 Simco Limited under exclusive license to ...,3:27,25-Nov-13,Better Than Words,One Direction,Midnight Memories ( Deluxe ),Pop,$ 1.29,"( C ) 2014 Disney Enterprises , Inc.",3:28,"November 25 , 2013"
324,0.131542,1,We Dem Boyz,Wiz Khalifa,Blacc Hollywood ( Deluxe Version ),"Hip-Hop/Rap , Music , Rap , Hardcore Rap , Eas...",Album Only,2014 Atlantic Recording Corporation for the Un...,3:56,19-Aug-14,We Dem Boyz [ Explicit ],Wiz Khalifa,Blacc Hollywood [ Explicit ],Rap & Hip-Hop,$ 1.29,2014 Atlantic Recording Corporation for the Un...,3:45,"August 19 , 2014"
280,0.303020,1,Whateva ( feat . Monty ),Fetty Wap,Fetty Wap ( Deluxe ),"Hip-Hop/Rap , Music , Rap",Album Only,2015 300 Entertainment/RGF Productions,5:11,NaN,Whateva ( feat . Monty ) [ Clean ],Fetty Wap,Fetty Wap ( Deluxe ) [ Clean ],Rap & Hip-Hop,$ 1.29,"( C ) 2000 Mute Records Ltd. , a BMG Company ,...",5:11,"September 25 , 2015"
339,0.463379,1,True Colors ( feat . Nicki Minaj ),Wiz Khalifa,Blacc Hollywood,"Hip-Hop/Rap , Music , Rap , East Coast Rap , H...",Album Only,2014 Atlantic Recording Corporation for the Un...,4:15,19-Aug-14,True Colors ( feat . Nicki Minaj ) [ Explicit ],Wiz Khalifa,Blacc Hollywood [ Explicit ],Rap & Hip-Hop,$ 1.29,2014 Atlantic Recording Corporation for the Un...,4:15,"August 19 , 2014"
370,0.173394,1,Sweet Spot 2.0 ( feat . Jennifer Lopez ),Flo Rida,Wild Ones,"Hip-Hop/Rap , Music , Rap , Electronic , World...",Album Only,2012 Atlantic Recording Corporation for the Un...,3:11,22-Jun-12,Sweet Spot ( feat . Jennifer Lopez ),Flo Rida,Wild Ones,"Dance & Electronic , Rap & Hip-Hop , House",$ 1.29,"( C ) 2015 MCA Nashville , a Division of UMG R...",3:48,"June 22 , 2012"
373,0.154814,1,"I Do n't Like It , I Love It ( feat . Robin Th...",Flo Rida,"I Do n't Like It , I Love It ( feat . Robin Th...","Dance , Music",Album Only,2015 Atlantic Recording Corporation for the Un...,3:19,10-Jul-15,"I Do n't Like It , I Love It ( feat . Robin Th...",Flo Rida,"I Do n't Like It , I Love It ( feat . Robin Th...",Dance & Electronic,$ 1.29,2015 Atlantic Recording Corporation for the Un...,4:27,"June 23 , 2015"
186,0.151712,1,Illusion ( feat . Echosmith ),Zedd,True Colors,"Dance , Music , Electronic",Album Only,2015 Interscope Records,6:30,18-May-15,Illusion [ feat . Echosmith ],Zedd,True Colors,Dance & Electronic,$ 1.29,( C ) 2015 Interscope Records,6:25,"May 18 , 2015"
389,0.136427,1,One Mississippi ( Live from CMT `` Listen Up '' ),Brett Eldredge,Bring You Back,"Country , Music , Honky Tonk , Traditional Cou...",Album Only,2013 Atlantic Recording Corporation for the Un...,4:10,6-Aug-13,One Mississippi,Brett Eldredge,Bring You Back,Country,$ 1.29,2011 Toraay Records,3:45,"August 6 , 2013"


In [162]:
new_preds_val[new_preds_val['match_score']<=0.5]

,match_score,label,left_Song_Name,left_Artist_Name,left_Album_Name,left_Genre,left_Price,left_CopyRight,left_Time,left_Released,right_Song_Name,right_Artist_Name,right_Album_Name,right_Genre,right_Price,right_CopyRight,right_Time,right_Released
id,,,,,,,,,,,,,,,,,,
48,0.996386,1,Afire Love,Ed Sheeran,x,"Singer/Songwriter , Music , Pop , Rock , Conte...",$ 1.99,"2014 Asylum Records UK , a Warner Music UK Com...",5:14,20-Jun-14,Afire Love,Ed Sheeran,x,Pop,$ 1.29,( C ) 2014 mau5trap Recordings Ltd,5:14,"June 20 , 2014"
215,0.995265,1,Jack Daniels and Jesus,Chase Rice,Ignite the Night,"Country , Music , Urban Cowboy , Contemporary ...",$ 1.99,2014 Dack Janiels Records,3:55,19-Aug-14,Jack Daniels and Jesus,Chase Rice,Ignite the Night ( Party Edition ) [ Explicit ],Country,$ 1.29,"( C ) 2014 Dack Janiels Records , under exclus...",3:55,"August 19 , 2014"
251,0.993300,1,I Ca n't Go There ( Acoustic Version ),Kenny Chesney,"No Shoes , No Shirt , No Problems","Country , Music , Contemporary Country , Urban...",$ 1.99,2002 BMG,3:45,2-Apr-02,I Ca n't Go There ( Acoustic Version ),Kenny Chesney,"No Shoes , No Shirt , No Problems",Country,$ 1.29,Underground Avenue,3:45,"April 2 , 2002"
522,0.847501,1,Space Bound,Eminem,Recovery ( Deluxe Edition ),"Hip-Hop/Rap , Music , Rock , Hardcore Rap , Rap",$ 1.99,‰ ãÑ 2010 Aftermath Records,4:38,21-Jun-10,Space Bound [ Explicit ],Eminem,Recovery ( Amazon Version ) [ Explicit ] [ + v...,"Gangsta & Hardcore , Rap & Hip-Hop",$ 1.29,( C ) 2010 Aftermath Records,4:39,"June 21 , 2010"
1,0.992991,1,Credit,Meghan Trainor,Title ( Deluxe ),"Pop , Music , Rock , Pop/Rock , Dance , Teen Pop",$ 1.99,"2014 , 2015 Epic Records , a division of Sony ...",2:51,9-Jan-15,Credit,Meghan Trainor,Title ( Deluxe ),Pop,$ 1.29,"2011 What A Music Ltd , licence exclusive Parl...",2:51,"January 9 , 2015"
54,0.994422,1,I Ai n't Livin ' Long Like This,Chris Janson,Take It to the Bank - EP,"Country , Music",$ 1.99,2014 Sony Music Entertainment,4:13,24-Feb-14,I Ai n't Livin ' Long Like This,Chris Janson,Take It to the Bank ( EP ),Country,$ 1.29,( C ) 2012 Cash Money Records Inc.,4:13,"February 24 , 2014"
276,0.820484,1,Whateva ( feat . Monty ),Fetty Wap,Fetty Wap ( Deluxe ),"Hip-Hop/Rap , Music , Rap",$ 1.99,2015 300 Entertainment/RGF Productions,5:11,NaN,Whateva ( feat . Monty ) [ Explicit ],Fetty Wap,Fetty Wap ( Deluxe ) [ Explicit ],Rap & Hip-Hop,$ 1.29,"( C ) 2000 Mute Records Ltd. , a BMG Company ,...",5:11,"September 25 , 2015"
207,0.987411,1,Fake ( Demo ),Shinedown,Us and Them ( Deluxe Version ),"Rock , Music , Metal , Hard Rock , Alternative",$ 1.99,2009 Atlantic Recording Corporation for the Un...,5:05,4-Oct-05,Fake [ Demo ],Shinedown,Us And Them ( Deluxe ) [ Explicit ],Rock,$ 1.29,2015 Real Sounds,5:05,"October 4 , 2005"
517,0.992405,1,Something Great,One Direction,Midnight Memories ( Deluxe Edition ),"Pop , Music , Rock , Teen Pop , Pop/Rock , Dance",$ 1.99,‰ ãÑ 2013 Simco Limited under exclusive licens...,3:56,25-Nov-13,Something Great,One Direction,Midnight Memories ( Deluxe ),Pop,$ 1.29,"( C ) 2014 Disney Enterprises , Inc.",3:56,"November 25 , 2013"
